## Creating a deterministic critic agent

The function below builds a multi-layer perceptron where the size of each layer is given in the `size` list. We also specify the activation function of neurons at each layer and optionally a different activation function for the final layer.

In [ ]:
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

The `DiscreteQAgent` class implements a critic such the one used in DQN. It has one output neuron per action and its output is the Q-value of these actions given the state.

As any BBRL agent, it has a forward function that takes a time state as input. This forward function outputs the Q-values at the corresponding time step. Additionally, if the critic is used to choose an action, it also outputs the chosen action at the same time step.

In [ ]:
class DiscreteQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.is_q_function = True
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [action_dim], activation=nn.ReLU()
        )

    def forward(self, t, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        q_values = self.model(obs).squeeze(-1)
        self.set(("q_values", t), q_values)
        if choose_action:
            action = q_values.argmax(1)
            self.set(("action", t), action)
